---
<p align="center">
  <img src="https://github.com/lacamposm/course-fundamentals-llms-openai-langchain/raw/main/images/image_igac.jpg" alt="Imagen_IGAC" width="280">
</p>

---

# ***Fundamentos de LLMs con Python: Explorando ChatGPT y LangChain***

---

#### ***Instructor: [Luis Andrés Campos Maldonado](https://www.linkedin.com/in/lacamposm/)***

##### ***Email: luisandres.campos@igac.gov.co***

##### ***Contratista-Observatorio Inmobiliario Catastral***





---

# ***Clase 06 - 05 de abril de 2024***
---

## ***Similarity Search y QA Retrieval Chain***

**Objetivos de Aprendizaje:**

- Explicar los principios de la búsqueda por similitud y su importancia en la recuperación de información.
- Introducir el concepto de QA Retrieval Chain y cómo se puede implementar para buscar respuestas en documentos.

En nuestra sesión anterior, exploramos los conceptos de:

- Embeddings.
- Bases de datos vectoriales.
- Similaridad.
- Recuperación de información relevante.

In [1]:
!pip install pandas openpyxl langchain openai langchain-openai langchain-community langchain-core langchain-text-splitters chromadb pypdf

In [2]:
# Definimos una variable de entorno para la API KEY de OpenAI.
import os
os.environ["OPENAI_API_KEY"] = "<YOUR_API_KEY>"

In [3]:
import requests
from langchain_community.document_loaders import TextLoader
from langchain_openai import OpenAIEmbeddings
from langchain_community.vectorstores import Chroma
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [4]:
response = requests.get(
    "https://raw.githubusercontent.com/lacamposm/course-fundamentals-llms-openai-langchain/main/data/soledad_fragmento.txt"
    )
with open("soledad_fragmento.txt", "w", encoding="utf-8") as file:
        file.write(response.text)

openai_embeddings = OpenAIEmbeddings(
    model="text-embedding-ada-002",
 )

vectorstore = Chroma(collection_name="final_soledad_fragmento")
loader = TextLoader("soledad_fragmento.txt")
documents = loader.load()
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=1000, chunk_overlap=100)
docs = text_splitter.split_documents(documents)
db = vectorstore.from_documents(documents=docs, embedding=openai_embeddings)

In [5]:
def get_text_relevant_soledad_fragmento(query, number_docs=2):
    """
    Realiza una búsqueda de los fragmentos de texto más relevantes respecto a una consulta específica
    utilizando una base de datos vectorial para encontrar las correspondencias semánticas más cercanas.

    Parámetros:
    - query (str): El texto de la consulta para la cual se buscan fragmentos relevantes.
    - number_docs (int, opcional): El número de documentos más relevantes que se deben retornar.
    """
    return db.similarity_search(query, k=number_docs)

get_text_relevant_soledad_fragmento("¿Cuando volvieron los gitanos?", number_docs=4)

[Document(page_content='\nCuando volvieron los gitanos, Úrsula había predispuesto contra ellos a toda la población. Pero la curiosidad pudo más que el temor, porque aquella vez los gitanos recorrieron la aldea haciendo un ruido ensordecedor con toda clase de instrumentos músicos, mientras el pregonero anunciaba la exhibición del más fabuloso hallazgo de los nasciancenos. De modo que todo el mundo se fue a la carpa, y mediante el pago de un centavo vieron un Melquíades juvenil, repuesto, desarrugado, con una dentadura nueva y radiante. Quienes recordaban sus encías destruidas por el escorbuto, sus mejillas fláccidas y sus labios marchitos se estremecieron de pavor ante aquella prueba terminante de los poderes sobrenaturales del gitano. El pavor se convirtió en pánico cuando Melquíades se sacó los dientes, intactos, engastados en las encías, y se los mostró al público por un instante —un instante fugaz en que volvió a ser el mismo hombre decrépito de los años anteriores— y se los puso ot

In [6]:
get_text_relevant_soledad_fragmento("¿Que es Macondo?", number_docs=2)

[Document(page_content='José Arcadio Buendía, que era el hombre más emprendedor que se vería jamás en la aldea, había dispuesto de tal modo la posición de las casas, que desde todas podía llegarse al río y abastecerse de agua con igual esfuerzo, y trazó las calles con tan buen sentido que ninguna casa recibía más sol que otra a la hora del calor. En pocos años, Macondo fue una aldea más ordenada y laboriosa que cualquiera de las conocidas hasta entonces por sus 300 habitantes. Era en verdad una aldea feliz, donde nadie era mayor de treinta años y donde nadie había muerto.', metadata={'source': 'soledad_fragmento.txt'}),
 Document(page_content='\nLa idea de un Macondo peninsular prevaleció durante mucho tiempo, inspirada en el mapa arbitrario que dibujó José Arcadio Buendía al regreso de su expedición. Lo trazó con rabia, exagerando de mala fe las dificultades de comunicación, como para castigarse a sí mismo por la absoluta falta de sentido con que eligió el lugar. «Nunca llegaremos a n

<p align="center">
  <img src="https://github.com/lacamposm/course-fundamentals-llms-openai-langchain/raw/main/images/retrieval_data_connection.jpg" width="900" height="350">
</p>

## ***RAG - Retriver Augmented Generation.***

Es un marco de trabajo en el procesamiento de lenguaje natural (NLP) diseñado para mejorar las tareas de pregunta y respuesta (QA) mediante la integración de un mecanismo de búsqueda (retrieval) con la generación de texto basada en LLMs. Este enfoque híbrido utiliza primero un componente de retrieval para identificar y recuperar fragmentos de texto relevantes de una base de datos o conjunto de documentos, y luego utiliza un componente de generation para sintetizar la información recuperada en una respuesta coherente y contextualmente rica. Veamos más de cerca los dos componentes principales:

- Retriever.

El retriever se encarga de buscar en una colección de documentos para encontrar aquellos que son más relevantes para una pregunta específica. Funciona identificando similitudes semánticas entre la pregunta y los documentos utilizando técnicas como la búsqueda por similitud en espacios vectoriales. El objetivo es filtrar la vasta cantidad de información disponible y seleccionar solo los fragmentos de texto que probablemente contengan la respuesta a la pregunta.

- Generator.

El generator es un LLM, este toma la información recuperada por el retriever y la procesa para generar una respuesta. Este componente tiene la capacidad de comprender el contexto proporcionado por los fragmentos de texto seleccionados y sintetizar una respuesta coherente y precisa que aborde la pregunta original.

- Funcionamiento de RAG.

La combinación de estos componentes permite que el RAG aborde preguntas complejas que requieren conocimiento especializado o específico más allá del contenido entrenado en los modelos de generación de lenguaje. Al acceder dinámicamente a información relevante durante el proceso de generación de respuestas, RAG es capaz de producir respuestas que son tanto informativas como específicas, reflejando una comprensión más profunda y actualizada del tema en cuestión.

- Aplicaciones de RAG.

RAG es particularmente útil en sistemas de QA donde la precisión y relevancia de las respuestas son críticas, como en asistentes virtuales, herramientas de soporte al cliente o en aplicaciones educativas. Al integrar la búsqueda y la generación de manera fluida, RAG representa un avance significativo en la capacidad de los sistemas de NLP para manejar consultas complejas y proporcionar respuestas que satisfacen las necesidades de información de los usuarios.

### ***Ejemplo***

Deseamos realizar preguntas al documento:

 *CATASTRO MULTIPROPÓSITO PARA LA PLANEACIÓN Y LA GESTIÓN TERRITORIAL DOCUMENTO CONCEPTUAL*

emitido por el *Departamento Nacional de Planeación DNP* en noviembre de 2023.

In [1]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.runnables import RunnablePassthrough
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_community.vectorstores import Chroma
from langchain_community.document_loaders import PyPDFLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter

In [8]:
llm = ChatOpenAI(temperature=0.2)
openai_embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")
loader = PyPDFLoader("DNP_catastro_multiproposito.pdf")  # Subir el archivo PDF a Google Colab.
pages = loader.load_and_split()
text_splitter = RecursiveCharacterTextSplitter(separators=["\n\n", "\n"], chunk_size=1000, chunk_overlap=100)
documents = text_splitter.split_documents(pages)
vectorstore = Chroma.from_documents(documents, embedding=openai_embeddings)
retriever = vectorstore.as_retriever()

In [9]:
llm.invoke("Que dice la  Ley  2294  de  2023  del  Plan  Nacional  de  Desarrollo de Colombia?")

AIMessage(content='Lo siento, pero no tengo información sobre la Ley 2294 de 2023 del Plan Nacional de Desarrollo de Colombia, ya que esta ley es ficticia y no existe en la realidad. Si necesitas información sobre alguna ley o tema específico, estaré encantado de intentar ayudarte.')

In [11]:
rag_prompt = """Eres un asistente especialista en catastro en el contexto del país Colombia\
Responde las preguntas basándote en el siguiente contexto:
{context}
Pregunta: {question}
"""
after_rag_prompt = ChatPromptTemplate.from_template(rag_prompt)

rag_chain = (
        {"context": retriever, "question": RunnablePassthrough()}
        | after_rag_prompt
        | llm
        | StrOutputParser()
)

In [12]:
rag_chain.invoke("Sobre que habla la Ley 2294 de 2023?")

'La Ley 2294 de 2023 del Plan Nacional de Desarrollo en Colombia habla sobre la consolidación del Catastro Multipropósito y el tránsito hacia el Sistema de Administración del Territorio (SAT) como factor clave para la gestión territorial en todas sus dimensiones, y para el desarrollo y la ejecución efectiva de estrategias y políticas relacionadas con el uso y la gestión del territorio y de los recursos en el país. Esta ley establece la importancia de contar con información actualizada y precisa del Catastro Multipropósito para la toma de decisiones en asuntos de planificación urbana y rural, así como para abordar desafíos relacionados con la tenencia de la tierra, la sostenibilidad ambiental, las finanzas públicas territoriales y el desarrollo económico.'

In [13]:
rag_chain.invoke("A que hace referencia el catastro Multipropósito como eje catalizador del ordenamiento del territorio alrededor del agua")

'El catastro Multipropósito como eje catalizador del ordenamiento del territorio alrededor del agua hace referencia a la integración de diferentes componentes ambientales, sociales, culturales y económicos en la dinámica territorial del país, con el objetivo de buscar la articulación y simplificación de los instrumentos de ordenamiento territorial. En este sentido, el catastro Multipropósito busca ubicar al agua como eje articulador de la interrelación y dinámica entre todos los elementos del territorio, con el fin de garantizar la estabilidad territorial y el funcionamiento adecuado de las actividades humanas en relación con el medio ambiente.'

In [14]:
rag_chain.invoke("¿Cuales son los principales objetivos del catastro multiproposito?")

'Los principales objetivos del catastro multipropósito son:\n1. Lograr la identificación física, jurídica y económica de los bienes inmuebles en el territorio nacional.\n2. Servir como insumo fundamental en la formulación e implementación de diversas políticas públicas y procesos de gestión territorial.\n3. Contribuir al desarrollo y ordenamiento territorial.\n4. Brindar mayor seguridad jurídica.\n5. Optimizar el mercado inmobiliario.'

In [15]:
rag_chain.invoke("¿Que papel desempeña el IGAC en el catastro multiproposito?. Dame la respuesta en máximo 6 bullets")

'- El IGAC es uno de los gestores catastrales que presta el servicio catastral en Colombia.\n- Es responsable de recoger la información sobre los derechos, restricciones y responsabilidades que recaen en el territorio.\n- Se articula con otros sistemas de información territorial para garantizar la integridad de los datos catastrales.\n- Participa en el fortalecimiento institucional y tecnológico del catastro multipropósito.\n- Inicia el levantamiento catastral en un primer grupo de municipios en colaboración con la SNR y la ANT.\n- Es el gestor catastral por excepción en caso de que el municipio decida contratar el servicio con otro gestor.'